In [1]:
"""
Test script for LAT detection module
"""
import sys

# Path to your repo (adjust this)
repo_path = "/Users/pamco116/Documents/GitHub/EMTO_input_automation/"

# Add it to sys.path if not already there
if repo_path not in sys.path:
    sys.path.append(repo_path)

from modules.lat_detector import get_emto_lattice_info, validate_emto_vectors

In [2]:
# Test with your existing FePt.cif
cif_file = 'FePt.cif'

info = get_emto_lattice_info(cif_file)

In [8]:
info = get_emto_lattice_info(cif_file)

print(f"\nResults:")
print(f"  File: {cif_file}")
print(f"  Detected LAT: {info['lat']}")
print(f"  Lattice name: {info['lattice_name']}")
print(f"  Crystal system: {info['crystal_system']}")
print(f"  Centering: {info['centering']}")

print(f"\n  Lattice parameters:")
print(f"    a = {info['a']:.6f} Å")
print(f"    b = {info['b']:.6f} Å")
print(f"    c = {info['c']:.6f} Å")
print(f"    alpha = {info['alpha']:.3f}°")
print(f"    beta  = {info['beta']:.3f}°")
print(f"    gamma = {info['gamma']:.3f}°")

print(f"\n  Ratios:")
print(f"    b/a = {info['boa']:.6f}")
print(f"    c/a = {info['coa']:.6f}")

print(f"\n  EMTO Primitive vectors (normalized to a):")
print(f"    BSX = [{info['BSX'][0]:8.5f}, {info['BSX'][1]:8.5f}, {info['BSX'][2]:8.5f}]")
print(f"    BSY = [{info['BSY'][0]:8.5f}, {info['BSY'][1]:8.5f}, {info['BSY'][2]:8.5f}]")
print(f"    BSZ = [{info['BSZ'][0]:8.5f}, {info['BSZ'][1]:8.5f}, {info['BSZ'][2]:8.5f}]")

print(f"\n  Number of atoms: {len(info['atoms'])}")
print(f"  Atom types: {[atom.symbol for atom in info['atoms']]}")


Results:
  File: FePt.cif
  Detected LAT: 5
  Lattice name: Simple tetragonal
  Crystal system: tetragonal
  Centering: P

  Lattice parameters:
    a = 2.728004 Å
    b = 2.728004 Å
    c = 3.737356 Å
    alpha = 90.000°
    beta  = 90.000°
    gamma = 90.000°

  Ratios:
    b/a = 1.000000
    c/a = 1.369997

  EMTO Primitive vectors (normalized to a):
    BSX = [ 1.00000,  0.00000,  0.00000]
    BSY = [ 0.00000,  1.00000,  0.00000]
    BSZ = [ 0.00000,  0.00000,  1.37000]

  Number of atoms: 2
  Atom types: ['Fe', 'Pt']


In [11]:
a = info['matrix']
a = a.copy()
a[2][2] = 43
a

array([[ 2.72800402e+00,  0.00000000e+00,  1.67042070e-16],
       [-1.67042070e-16,  2.72800402e+00,  1.67042070e-16],
       [ 0.00000000e+00,  0.00000000e+00,  4.30000000e+01]])

In [9]:

# Test 2: Validate vectors
print("\n" + "="*70)
print("2. Validating EMTO vectors against pymatgen...")
print("="*70)

try:
    is_valid = validate_emto_vectors(cif_file, verbose=True)
    
    if is_valid:
        print("✓ Test 2 PASSED - Volumes match!")
    else:
        print("✗ Test 2 FAILED - Volume mismatch!")
        
except Exception as e:
    print(f"\n✗ Test 2 FAILED")
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

print("\n" + "="*70)
print("Testing complete!")
print("="*70)


2. Validating EMTO vectors against pymatgen...

Validation for: FePt.cif
Detected: LAT=5 (Simple tetragonal)
Crystal system: tetragonal, Centering: P

Pymatgen volume: 27.813428 Å³
EMTO volume:     27.813428 Å³
Difference:      0.000000e+00 Å³

Lattice parameters:
  a = 2.728004 Å
  b = 2.728004 Å  (b/a = 1.000000)
  c = 3.737356 Å  (c/a = 1.369997)

Primitive vectors (normalized to a):
  BSX = [ 1.00000,  0.00000,  0.00000]
  BSY = [ 0.00000,  1.00000,  0.00000]
  BSZ = [ 0.00000,  0.00000,  1.37000]

Validation: ✓ PASS

✓ Test 2 PASSED - Volumes match!

Testing complete!


In [10]:
"""
Test KSTR generation with auto-detected LAT
"""

from modules.inputs.kstr_from_cif import create_kstr_input_from_cif
import os

# Test with FePt.cif
cif_file = 'FePt.cif'
output_path = 'test_output'
job_name = 'fept_test'
dmax = 1.3

print("="*70)
print("Testing KSTR generation with auto-detected LAT")
print("="*70)

# Test 1: Auto-detect LAT (don't provide lat parameter)
print("\n1. Testing with auto-detected LAT...")
try:
    result = create_kstr_input_from_cif(
        cif_file=cif_file,
        output_path=output_path,
        job_name=job_name,
        dmax=dmax
        # Note: lat=None is default, so we don't specify it
    )
    
    print(f"\nResults:")
    print(f"  LAT: {result['lat']} ({result['lattice_name']})")
    print(f"  NL: {result['NL']}")
    print(f"  NQ3: {result['NQ3']}")
    print(f"  Atoms: {result['atoms']}")
    print(f"  a, b, c: {result['a']:.4f}, {result['b']:.4f}, {result['c']:.4f}")
    
    # Check if output file was created
    output_file = f"{output_path}/smx/{job_name}.dat"
    if os.path.exists(output_file):
        print(f"\n✓ Output file created: {output_file}")
        print("\n✓ Test 1 PASSED")
    else:
        print(f"\n✗ Output file not found: {output_file}")
        print("✗ Test 1 FAILED")
        
except Exception as e:
    print(f"\n✗ Test 1 FAILED")
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

# Test 2: Manually specify LAT (should show warning if different)
print("\n" + "="*70)
print("2. Testing with manually specified LAT=5...")
print("="*70)

try:
    result = create_kstr_input_from_cif(
        cif_file=cif_file,
        output_path=output_path,
        job_name=job_name + "_manual",
        dmax=dmax,
        lat=10  # Manually specify
    )
    
    print(f"\nResults:")
    print(f"  LAT: {result['lat']} ({result['lattice_name']})")
    print("\n✓ Test 2 PASSED")
    
except Exception as e:
    print(f"\n✗ Test 2 FAILED")
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

print("\n" + "="*70)
print("Testing complete!")
print("="*70)

Testing KSTR generation with auto-detected LAT

1. Testing with auto-detected LAT...
Auto-detected LAT=5 (Simple tetragonal)
Auto-determined NL=3 from electronic structure
KSTR input file 'test_output/smx/fept_test.dat' created successfully.

Results:
  LAT: 5 (Simple tetragonal)
  NL: 3
  NQ3: 2
  Atoms: ['Fe', 'Pt']
  a, b, c: 2.7280, 2.7280, 3.7374

✓ Output file created: test_output/smx/fept_test.dat

✓ Test 1 PASSED

2. Testing with manually specified LAT=5...
Using provided LAT=10
Auto-determined NL=3 from electronic structure
KSTR input file 'test_output/smx/fept_test_manual.dat' created successfully.

Results:
  LAT: 10 (Simple tetragonal)

✓ Test 2 PASSED

Testing complete!


In [2]:
"""
Example: Using EMTO workflow with CIF files and LAT auto-detection
"""

from modules.workflows import create_emto_inputs

# Example 1: Full auto-detection (recommended for new workflows)
create_emto_inputs(
    output_path="./calculations/fept",
    job_name="fept",
    dmax=1.3,
    lat=None,  # Auto-detect from CIF
    ca_ratios=[0.92, 0.96, 1.00, 1.04],
    sws_values=[2.60, 2.65, 2.70],
    from_cif=True,
    cif_file="FePt.cif",
    create_job_script=True,
    job_mode='serial'
)

# Example 2: Manual LAT override (if you want to force a specific LAT)
create_emto_inputs(
    output_path="./calculations/fept_manual",
    job_name="fept",
    dmax=1.3,
    lat=10,  # Manually specify
    ca_ratios=[0.96, 1.00],
    sws_values=[2.60, 2.65],
    from_cif=True,
    cif_file="FePt.cif",
    create_job_script=False
)

Created directory structure in: ./calculations/fept

Creating input files for 4 c/a ratios and 3 SWS values...

  c/a = 0.92
    LAT will be auto-detected from CIF
Auto-detected LAT=5 (Simple tetragonal)
Auto-determined NL=3 from electronic structure
KSTR input file './calculations/fept/smx/fept_0.92.dat' created successfully.
SHAPE input file './calculations/fept/shp/fept_0.92.dat' created successfully.
KGRN input file './calculations/fept/fept_0.92_2.60.dat' created successfully.
KFCD input file './calculations/fept/fcd/fept_0.92_2.60.dat' created successfully.
KGRN input file './calculations/fept/fept_0.92_2.65.dat' created successfully.
KFCD input file './calculations/fept/fcd/fept_0.92_2.65.dat' created successfully.
KGRN input file './calculations/fept/fept_0.92_2.70.dat' created successfully.
KFCD input file './calculations/fept/fcd/fept_0.92_2.70.dat' created successfully.

  c/a = 0.96
    LAT will be auto-detected from CIF
Auto-detected LAT=5 (Simple tetragonal)
Auto-determin

In [2]:
"""
Comprehensive test of the complete LAT auto-detection implementation
"""

import numpy as np
import os
from modules.lat_detector import get_emto_lattice_info
from modules.inputs.kstr_from_cif import create_kstr_input_from_cif
from modules.workflows import create_emto_inputs
import shutil

print("="*70)
print("COMPREHENSIVE TEST: LAT Auto-Detection Implementation")
print("="*70)

cif_file = "FePt.cif"

# ============================================================
# TEST 1: Check lat_detector returns fractional_coords
# ============================================================
print("\n" + "="*70)
print("TEST 1: lat_detector.get_emto_lattice_info()")
print("="*70)

try:
    info = get_emto_lattice_info(cif_file)
    
    print(f"\n✓ LAT detected: {info['lat']} ({info['lattice_name']})")
    print(f"✓ Crystal system: {info['crystal_system']}, Centering: {info['centering']}")
    print(f"✓ Lattice params: a={info['a']:.4f}, b={info['b']:.4f}, c={info['c']:.4f}")
    print(f"✓ Ratios: b/a={info['boa']:.4f}, c/a={info['coa']:.4f}")
    
    # Check fractional_coords exists
    if 'fractional_coords' in info:
        print(f"✓ Fractional coords returned: shape {info['fractional_coords'].shape}")
        print(f"  First atom: {info['fractional_coords'][0]}")
    else:
        print("✗ FAIL: fractional_coords not in return dictionary!")
        raise KeyError("fractional_coords missing")
    
    print("\n✓ TEST 1 PASSED")
    
except Exception as e:
    print(f"\n✗ TEST 1 FAILED: {e}")
    import traceback
    traceback.print_exc()
    exit(1)

# ============================================================
# TEST 2: Check kstr_from_cif with ca_ratio parameter
# ============================================================
print("\n" + "="*70)
print("TEST 2: create_kstr_input_from_cif() with ca_ratio")
print("="*70)

test_output = "testing/test_kstr_ca"
if os.path.exists(test_output):
    shutil.rmtree(test_output)

try:
    # Test with ca_ratio=0.96
    result = create_kstr_input_from_cif(
        cif_file=cif_file,
        output_path=test_output,
        job_name="fept_096",
        dmax=1.3,
        ca_ratio=0.96,
        lat=None  # Auto-detect
    )
    
    print(f"\n✓ KSTR created with ca_ratio=0.96")
    print(f"  LAT: {result['lat']} ({result['lattice_name']})")
    print(f"  NL: {result['NL']}")
    print(f"  NQ3: {result['NQ3']}")
    
    # Check the file content
    kstr_file = os.path.join(test_output, "smx/fept_096.dat")
    if os.path.exists(kstr_file):
        with open(kstr_file, 'r') as f:
            lines = f.readlines()
            # Find lattice vectors (after the A/B/C line)
            for i, line in enumerate(lines):
                if 'A........=' in line:
                    print(f"\n  Lattice parameters line:")
                    print(f"    {line.strip()}")
                    print(f"\n  Lattice vectors:")
                    print(f"    BSX: {lines[i+1].strip()}")
                    print(f"    BSY: {lines[i+2].strip()}")
                    print(f"    BSZ: {lines[i+3].strip()}")
                    
                    # Parse BSZ to check z-component
                    bsz_values = lines[i+3].split()
                    bsz_z = float(bsz_values[-1])
                    
                    if abs(bsz_z - 0.96) < 0.01:
                        print(f"\n✓ BSZ z-component = {bsz_z:.6f} (expected ~0.96)")
                    else:
                        print(f"\n✗ BSZ z-component = {bsz_z:.6f} (expected ~0.96)")
                        print("  Warning: Scaling may not be working correctly")
                    break
        
        print(f"\n✓ KSTR file created: {kstr_file}")
    else:
        print(f"\n✗ KSTR file not found: {kstr_file}")
    
    print("\n✓ TEST 2 PASSED")
    
except Exception as e:
    print(f"\n✗ TEST 2 FAILED: {e}")
    import traceback
    traceback.print_exc()
    exit(1)

# ============================================================
# TEST 3: Full workflow with multiple ca_ratios
# ============================================================
print("\n" + "="*70)
print("TEST 3: Complete workflow with ca_ratio sweep")
print("="*70)

workflow_output = "testing/test_workflow_final"
if os.path.exists(workflow_output):
    shutil.rmtree(workflow_output)

try:
    create_emto_inputs(
        output_path=workflow_output,
        job_name="fept",
        dmax=1.3,
        lat=None,  # Auto-detect
        ca_ratios=[0.96, 1.00],
        sws_values=[2.60, 2.65],
        from_cif=True,
        cif_file=cif_file,
        create_job_script=False
    )
    
    print("\n✓ Workflow completed")
    
    # Verify files
    print("\nVerifying created files:")
    test_files = [
        "smx/fept_0.96.dat",
        "smx/fept_1.00.dat",
    ]
    
    all_ok = True
    for fname in test_files:
        fpath = os.path.join(workflow_output, fname)
        if os.path.exists(fpath):
            print(f"  ✓ {fname}")
            
            # Check lattice vectors
            with open(fpath, 'r') as f:
                lines = f.readlines()
                for i, line in enumerate(lines):
                    if 'A........=' in line:
                        bsz_line = lines[i+3]
                        bsz_z = float(bsz_line.split()[-1])
                        
                        # Extract expected ratio from filename
                        if '0.96' in fname:
                            expected = 0.96
                        elif '1.00' in fname:
                            expected = 1.00
                        
                        if abs(bsz_z - expected) < 0.01:
                            print(f"    ✓ c/a = {bsz_z:.6f} (expected {expected})")
                        else:
                            print(f"    ✗ c/a = {bsz_z:.6f} (expected {expected})")
                            all_ok = False
                        break
        else:
            print(f"  ✗ {fname} - MISSING")
            all_ok = False
    
    if all_ok:
        print("\n✓ TEST 3 PASSED - All files correct!")
    else:
        print("\n✗ TEST 3 FAILED - Some issues found")
    
except Exception as e:
    print(f"\n✗ TEST 3 FAILED: {e}")
    import traceback
    traceback.print_exc()
    exit(1)

# ============================================================
# SUMMARY
# ============================================================
print("\n" + "="*70)
print("ALL TESTS PASSED!")
print("="*70)
print("\nImplementation Summary:")
print("  ✓ LAT auto-detection from CIF")
print("  ✓ Fractional coordinates extraction")
print("  ✓ EMTO primitive vectors generation")
print("  ✓ c/a ratio scaling for sweeps")
print("  ✓ Full workflow integration")
print("\nYour implementation is complete and working!")
print("="*70)

COMPREHENSIVE TEST: LAT Auto-Detection Implementation

TEST 1: lat_detector.get_emto_lattice_info()

✓ LAT detected: 5 (Simple tetragonal)
✓ Crystal system: tetragonal, Centering: P
✓ Lattice params: a=2.7280, b=2.7280, c=3.7374
✓ Ratios: b/a=1.0000, c/a=1.3700
✓ Fractional coords returned: shape (2, 3)
  First atom: [0. 0. 0.]

✓ TEST 1 PASSED

TEST 2: create_kstr_input_from_cif() with ca_ratio
Auto-detected LAT=5 (Simple tetragonal)
Auto-determined NL=3 from electronic structure
KSTR input file 'testing/test_kstr_ca/smx/fept_096.dat' created successfully.

✓ KSTR created with ca_ratio=0.96
  LAT: 5 (Simple tetragonal)
  NL: 3
  NQ3: 2

  Lattice parameters line:
    A........= 1.0000000 B.......= 1.0000000 C.......= 0.9600000

  Lattice vectors:
    BSX: 1.00000000	0.00000000	0.00000000
    BSY: 0.00000000	1.00000000	0.00000000
    BSZ: 0.00000000	0.00000000	0.96000000

✓ BSZ z-component = 0.960000 (expected ~0.96)

✓ KSTR file created: testing/test_kstr_ca/smx/fept_096.dat

✓ TEST 2